In [1]:
import matplotlib.pyplot as plt
import numpy as np
import gvar
import pprint

In [2]:
import re
def parse_data(input_data):
    data = {}

    baryon_order = ["N_E0", "Xi_EO", "Sigma_E0","Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"]
    baryon_mapping = {'lambda_z': 'Lambda_E0',
                      'proton': 'N_E0',
                      'sigma_p': 'Sigma_E0',
                      'sigma_star_p': 'Sigma*_E0',
                      'xi_star_z': 'Xi*_E0',
                      'xi_z': 'Xi_EO',
                      'delta_pp':'Delta_E0'}

    for key, value in input_data.items():
        ensemble_name, baryon = key

        best = value['best']
        avg = value['avg']
        
        # number_best, uncertainty_best = map(float, re.match(r'(\d+\.\d+)\((\d+)\)', best).groups())
        # number_avg, uncertainty_avg = map(float, re.match(r'(\d+\.\d+)\((\d+)\)', avg).groups())

        number_best, uncertainty_best = gvar.mean(gvar.gvar(best)), gvar.sdev(gvar.gvar(best))
        number_avg, uncertainty_avg = gvar.mean(gvar.gvar(avg)), gvar.sdev(gvar.gvar(avg))

        if ensemble_name not in data:
            data[ensemble_name] = {'nolan_best':{},'nolan_avg':{}}

        new_baryon_name = baryon_mapping[baryon]
        
        data[ensemble_name]['nolan_best'][new_baryon_name] = (number_best, uncertainty_best)
        data[ensemble_name]['nolan_avg'] [new_baryon_name] = (number_avg, uncertainty_avg)
        
    for ensemble_name in data:
        for key in ['nolan_best','nolan_avg']:
            data[ensemble_name][key] = {baryon: data[ensemble_name][key].get(
                baryon, (None, None)) for baryon in baryon_order}

    return data


In [8]:
nolan = {}
nolan = gvar.load('nolan.p')
nolan_ = parse_data(nolan)  
pprint.pprint(nolan_)

{'a06m220L': {'nolan_avg': {'Delta_E0': (0.386, 0.015),
                            'Lambda_E0': (0.33340000000000003, 0.0022),
                            'N_E0': (0.29450000000000004, 0.0035),
                            'Sigma*_E0': (0.4188, 0.0079),
                            'Sigma_E0': (0.3524, 0.0028),
                            'Xi*_E0': (0.45030000000000003,
                                       0.0045000000000000005),
                            'Xi_EO': (0.383, 0.0012000000000000001)},
              'nolan_best': {'Delta_E0': (0.379, 0.015),
                             'Lambda_E0': (0.3325, 0.0019),
                             'N_E0': (0.29650000000000004,
                                      0.0018000000000000002),
                             'Sigma*_E0': (0.41650000000000004, 0.0073),
                             'Sigma_E0': (0.3518, 0.0021000000000000003),
                             'Xi*_E0': (0.45, 0.0038),
                             'Xi_EO': (0.38280000000000

In [9]:
import gvar
import re

def parse_latex_table(file_name):
    with open(file_name, "r") as f:
        content = f.read()

    tables = re.split(r'\\end{tabular}', content)[:-1]

    data = {}
    for table in tables:
        lines = table.strip().split("\n")
        if len(lines) < 4:  # If there are not enough lines, skip this table
            continue
        ensemble = lines[2].split("&")[0].strip().replace('$_','_').replace('$', '').replace('{', '').replace('}', '')
        
        data[ensemble] = {}

        for line in lines[3:-1]:  # Skip the first two lines (headers) and the last line (end of the table)
            cols = line.split("&")

            person = cols[0].strip()
            # Remove extra spaces within parentheses and then change the data to gvar
            results = [gvar.gvar(item.replace(" ", "").replace("{", "").replace("}", "")) if "(" in item else (None, None) for item in cols[1:]]
            results_dict = dict(zip(["N_E0", "Xi_EO", "Sigma_E0", "Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"], results))
            # Sorting the person's data and storing it as a list of tuples
            sorted_results = sorted(results_dict.items(), key=lambda item: item[1][0] if isinstance(item[1], tuple) and item[1][0] is not None else (item[1] if isinstance(item[1], gvar._gvarcore.GVar) else float('inf')))

            data[ensemble][person] = sorted_results

    return data

file_name = "tables.txt"
baryon_order = ["N_E0", "Xi_EO", "Sigma_E0","Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"]
data = parse_latex_table(file_name)

for ensemble_name in data:
    nolan_best = [(k, "{}({})".format(v[0], v[1]) if v[0] is not None and v[1] is not None else "None(None)") for k, v in nolan_[ensemble_name]['nolan_best'].items()]
    nolan_avg = [(k, "{}({})".format(v[0], v[1]) if v[0] is not None and v[1] is not None else "None(None)") for k, v in nolan_[ensemble_name]['nolan_avg'].items()]

# Skip 'None(None)' values
    data[ensemble_name]['nolan_best'] = sorted([(k, gvar.gvar(v)) for k, v in nolan_best if v != "None(None)"], key=lambda item: item[1].mean if item[1] is not None else float('inf'))
    data[ensemble_name]['nolan_avg'] = sorted([(k, gvar.gvar(v)) for k, v in nolan_avg if v != "None(None)"], key=lambda item: item[1].mean if item[1] is not None else float('inf'))

pprint.pprint(data)

{'a06m220L': {'Aaron': [('N_E0', 0.2969(18)),
                        ('Lambda_E0', 0.3360(11)),
                        ('Sigma_E0', 0.3560(12)),
                        ('Xi_EO', 0.38497(69)),
                        ('Delta_E0', 0.3889(96)),
                        ('Sigma*_E0', 0.4222(53)),
                        ('Xi*_E0', 0.4527(33))],
              'Amy': [('N_E0', 0.2925(41)),
                      ('Lambda_E0', 0.3335(19)),
                      ('Sigma_E0', 0.3538(22)),
                      ('Xi_EO', 0.38361(98)),
                      ('Delta_E0', 0.3943(90)),
                      ('Sigma*_E0', 0.4239(54)),
                      ('Xi*_E0', 0.4541(30))],
              'Daniel': [('N_E0', 0.2986(19)),
                         ('Lambda_E0', 0.3373(13)),
                         ('Sigma_E0', 0.3562(14)),
                         ('Xi_EO', 0.38548(91)),
                         ('Delta_E0', 0.4019(37)),
                         ('Sigma*_E0', 0.4316(22)),
                      

In [5]:
# import re  
# import pprint

# import gvar

# def parse_latex_table(file_name):
#     with open(file_name, "r") as f:
#         content = f.read()

#     tables = re.split(r'\\end{tabular}', content)[:-1]

#     data = {}
#     for table in tables:
#         lines = table.strip().split("\n")
#         if len(lines) < 4:  # If there are not enough lines, skip this table
#             continue
#         ensemble = lines[2].split("&")[0].strip().replace('$_','_').replace('$', '').replace('{', '').replace('}', '')
        
#         data[ensemble] = {}

#         for line in lines[3:-1]:  # Skip the first two lines (headers) and the last line (end of the table)
#             cols = line.split("&")

#             person = cols[0].strip()
#             # Here change the data to gvar
#             results = [tuple(map(gvar.gvar, re.findall(r"([\d.]+\(\d+\))", item))) if re.findall(r"([\d.]+\(\d+\))", item) else (None, None) for item in cols[1:]]
#             results_dict = dict(zip(["N_E0", "Xi_EO", "Sigma_E0", "Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"], results))
#             # Sorting the person's data and storing it as a list of tuples
#             sorted_results = sorted(results_dict.items(), key=lambda item: item[1][0] if item[1][0] is not None else float('inf'))
#             data[ensemble][person] = sorted_results

#     return data


# file_name = "tables.txt"
# baryon_order = ["N_E0", "Xi_EO", "Sigma_E0","Lambda_E0", "Xi*_E0", "Delta_E0", "Sigma*_E0"]
# data = parse_latex_table(file_name)
# for ensemble_name in data:
# # Sorting Nolan's data and storing it as a list of tuples
#     # data[ensemble_name]['nolan_best'] = sorted(nolan_[ensemble_name]['nolan_best'].items(), key=lambda item: item[1][0] if item[1][0] is not None else float('inf'))
#     # data[ensemble_name]['nolan_avg'] = sorted(nolan_[ensemble_name]['nolan_avg'].items(), key=lambda item: item[1][0] if item[1][0] is not None else float('inf'))
#     for ensemble_name in data:
#     # Sorting Nolan's data and storing it as a list of tuples
#         data[ensemble_name]['nolan_best'] = sorted([(k, gvar.gvar(v)) for k, v in nolan_[ensemble_name]['nolan_best'].items()], key=lambda item: item[1].mean if item[1] is not None else float('inf'))
#         data[ensemble_name]['nolan_avg'] = sorted([(k, gvar.gvar(v)) for k, v in nolan_[ensemble_name]['nolan_avg'].items()], key=lambda item: item[1].mean if item[1] is not None else float('inf'))

# pprint.pprint(data)

In [10]:

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

def plot_ensemble(ensemble_data, ensemble_name, pdf):
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_title(ensemble_name)

    offset = 0.08
    all_baryon_keys = []
    positions = {}

    for i, (person, results) in enumerate(ensemble_data.items()):
        if person == '\\hline' or person == 'Jinchen':
            continue
        # valid_results = {k: v for k, v in results if len(v) > 1 and v[1] is not None}
        valid_results = {k: v for k, v in results if v is not None}

        # print(valid_results)

        x = np.arange(len(valid_results))
        y  = []
        for result in valid_results.items():
            try:
                y.append(gvar.mean(result[1]))
            except ValueError:
                y.append(np.nan)
        # y = [gv.mean(result[1]) for result in valid_results.items() if gv.mean(result[1]) is not None]
        def truncate(number, significant_figures):
            if number == 0:
                return 0
            else:
                return round(number, significant_figures - int(np.floor(np.log10(abs(number)))) - 1)

        # yerr = [truncate(result[1] * 0.001,2) if result[1][1] is not None else 0 for result in valid_results.items()]
        yerr = []
        for result in valid_results.items():
            try:
                yerr.append(gvar.sdev(result[1]))
            except ValueError:
                yerr.append(np.nan)

        # yerr = [gv.sdev(result[1]) if gv.sdev(result[1]) is not None else 0 for result in valid_results.items()]

        print(yerr)
        # ax.errorbar(x + i * offset, y, yerr=yerr, fmt="o", capsize=5, label=person)
        ax.errorbar(x + i * offset, y, yerr=yerr, fmt="o", capsize=5, label=person)

        # Store the positions of each baryon for this person
        for j, key in enumerate(valid_results.keys()):
            if key in positions:
                positions[key].append(j + i * offset)
            else:
                positions[key] = [j + i * offset]

        all_baryon_keys += [k for k in valid_results.keys() if k not in all_baryon_keys]

    mean_positions = [np.mean(positions[key]) for key in all_baryon_keys]

    ax.set_xticks(mean_positions)
    ax.set_xticklabels(all_baryon_keys, rotation=45)
    ax.legend()
    pdf.savefig(fig, bbox_inches="tight")
    plt.close(fig)

with PdfPages("output_plots.pdf") as pdf:
    for ensemble_name, ensemble_data in data.items():
        plot_ensemble(ensemble_data, ensemble_name, pdf)


[0.0041, 0.0019, 0.0022, 0.0009800000000000002, 0.009000000000000001, 0.0054, 0.003]
[0.0017000000000000001, 0.0015, 0.0016, 0.0009000000000000001, 0.002, 0.0043, 0.0025]
[0.0018000000000000002, 0.0011, 0.0012000000000000001, 0.0006900000000000001, 0.009600000000000001, 0.0053, 0.0033]
[0.0019, 0.0013000000000000002, 0.0014, 0.0009100000000000001, 0.0037, 0.0022, 0.0015]
[0.0018000000000000002, 0.0019, 0.0021000000000000003, 0.015, 0.0011, 0.0073, 0.0038]
[0.0035, 0.0022, 0.0028, 0.0012000000000000001, 0.015, 0.0079, 0.0045000000000000005]
[0.0016, 0.0012000000000000001, 0.0018000000000000002, 0.007, 0.0009800000000000002, 0.0038, 0.0026000000000000003]
[0.0011, 0.0007300000000000001, 0.0008300000000000001, 0.0085, 0.00054, 0.0081, 0.0051]
[0.0011, 0.00079, 0.024, 0.0009000000000000001, 0.0006100000000000001, 0.009000000000000001, 0.0039000000000000003]
[0.0013000000000000002, 0.0009400000000000001, 0.001, 0.00071, 0.0028, 0.0019, 0.0013000000000000002]
[0.0012000000000000001, 0.025, 0

/usr/lib/python3.11/site-packages/matplotlib/axes/_axes.py:1185: RuntimeWarning: All-NaN axis encountered
  miny = np.nanmin(masked_verts[..., 1])
/usr/lib/python3.11/site-packages/matplotlib/axes/_axes.py:1186: RuntimeWarning: All-NaN axis encountered
  maxy = np.nanmax(masked_verts[..., 1])


[0.0088, 0.0048000000000000004, 0.0071, 0.0056, 0.0079, 0.0055000000000000005, 0.006900000000000001]
[0.014, 0.016, 0.017, 0.0028, 0.003, 0.0021000000000000003, 0.0016]
[0.0031000000000000003, 0.0018000000000000002, 0.0017000000000000001, 0.0012000000000000001, 0.0036000000000000003, 0.0022, 0.0016]
[0.0032, 0.0019, 0.0017000000000000001, 0.0012000000000000001, 0.0039000000000000003, 0.0025, 0.0017000000000000001]
[0.01, 0.0081, 0.007200000000000001, 0.004200000000000001, 0.0022, 0.0017000000000000001, 0.0013000000000000002]
[0.012, 0.01, 0.013000000000000001, 0.01, 0.0092, 0.0055000000000000005, 0.009000000000000001]
[0.007200000000000001, 0.0031000000000000003, 0.0043, 0.0017000000000000001, 0.0033, 0.0035, 0.0025]
[0.0031000000000000003, 0.0015, 0.0016, 0.0013000000000000002, 0.0033, 0.0024000000000000002, 0.0017000000000000001]
[0.0025, 0.0017000000000000001, 0.0018000000000000002, 0.0012000000000000001, 0.014, 0.0058000000000000005, 0.0023]
[0.0036000000000000003, 0.00210000000000